In [1]:
from io import StringIO

import pandas as pd
import requests

In [2]:
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
r = requests.get(url)

f = StringIO(r.text)
f.seek(0)
df = pd.read_csv(f)
f.close()

In [10]:
df_latest = df[df.date=='2021-04-20']

In [15]:
df_latest[df_latest['location'] == 'World']

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
81862,OWID_WRL,NaN,World,2021-04-20,142945718.0,847298.0,786128.286,3043955.0,14140.0,12113.857,...,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,0.737
